# try grid search

In [1]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
svr = svm.SVC()
clf = GridSearchCV(svr, params_svm, n_jobs=-1, cv=5, scoring='f1_macro')
clf.fit(iris.data, iris.target)

NameError: name 'params_svm' is not defined

In [63]:
def CV_models(models, n_jobs=-1, cv=5):
    cv_results = []
    for model, tuning_params, name, reg_or_class in models:
        print 'CVing', name, reg_or_class
        if reg_or_class == "regression":
            clf = GridSearchCV(model, tuning_params, n_jobs=n_jobs, cv=cv, scoring='neg_mean_squared_error')
        else:
            clf = GridSearchCV(model, tuning_params, n_jobs=n_jobs, cv=cv, scoring='f1_macro')

        clf.fit(iris.data, iris.target)
        df = pd.DataFrame(clf.cv_results_).sort('mean_test_score', ascending=False)

        test_score, train_score, fit_time = df[['mean_test_score', 'mean_train_score', 'mean_fit_time']].values[0]
        test_score = -1. * test_score
        train_score = -1. * train_score
        print 'best test mse = %.3f, train mse = %.3f, mean_fit_time = %f' %(test_score, train_score, fit_time)
        print 'best params', clf.best_params_
        print 

        result = {}
        result['model_name'] = name
        result['reg_or_cla'] = reg_or_class
        result['mean_test_mse'] = test_score
        result['mean_train_mse'] = train_score
        result['mean_fit_time'] = fit_time
        result['best_params'] = clf.best_params_
        result['best_model'] = clf.best_estimator_
        cv_results.append(result)
    return cv_results

In [64]:
models = get_models()
cv_results = CV_models(models[:2])
df_cv = pd.DataFrame(cv_results)
df_cv

CVing linear regression
best test mse = 0.069, train mse = 0.044, mean_fit_time = 0.087200
best params {}

CVing ridge regression
best test mse = 0.071, train mse = 0.044, mean_fit_time = 0.060200
best params {'alpha': 1.0}



,best_model,best_params,mean_fit_time,mean_test_mse,mean_train_mse,model_name,reg_or_cla
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",{},0.0872,0.069046,0.044180,linear,regression
1,"Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...",{u'alpha': 1.0},0.0602,0.070631,0.044479,ridge,regression


In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.4, random_state=0)

clf = linear_model.Lasso().fit(X_train, y_train)
clf.score(X_test, y_test)  

0.4355189856348346

In [49]:
mean_squared_error(y_test,clf.predict(X_test))

0.34417661848098274

In [44]:
for x in zip(clf.predict(X_test), y_test):
    print x[0], x[1]

1.15107186849 2
0.994199821093 1
0.62341134543 0
1.3222050111 2
0.637672440648 0
1.27942172545 2
0.609150250212 0
1.09402748762 1
1.10828858284 1
0.994199821093 1
1.22237734458 2
1.06550529718 1
1.09402748762 1
1.0797663924 1
1.09402748762 1
0.637672440648 0
1.06550529718 1
1.05124420196 1
0.62341134543 0
0.609150250212 0
1.12254967805 2
1.06550529718 1
0.694716821519 0
0.62341134543 0
1.10828858284 2
0.566366964559 0
0.694716821519 0
1.03698310675 1
0.894372154569 1
0.651933535866 0
1.20811624936 2
1.06550529718 1
0.637672440648 0
1.12254967805 2
1.22237734458 2
0.979938725875 1
0.666194631083 0
1.15107186849 1
1.02272201153 1
0.994199821093 1
1.20811624936 2
0.609150250212 0
1.15107186849 2
0.651933535866 0
0.637672440648 0
0.922894345004 1
1.20811624936 2
1.2366384398 2
1.13681077327 2
1.25089953501 2
0.979938725875 1
1.29368282067 2
1.09402748762 1
0.965677630658 1
1.12254967805 2
1.15107186849 2
1.06550529718 2
1.13681077327 2
1.09402748762 1
1.16533296371 2


In [38]:
y_test

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 0,
       0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1,
       2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2])

In [8]:
from sklearn import linear_model, svm, tree, ensemble
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
import geopandas as gp
from sklearn.metrics import classification_report

In [9]:
def fit_pred(model, train_dummies_x, train_y, test_dummies_x, test_y, regression=True):
    def round_reg_pred(pred):
        pred_round = pred.round()
        pred_round[pred_round<1] = 1
        pred_round[pred_round>4] = 4
        return pred_round
    
    model.fit(train_dummies_x, train_y)

    train_pred = model.predict(train_dummies_x)
    test_pred = model.predict(test_dummies_x)
    train_pred_round = round_reg_pred(train_pred) if regression else train_pred
    test_pred_round = round_reg_pred(test_pred) if regression else test_pred
    
    if regression:
        mse_train = mean_squared_error(train_y, train_pred)
        mse_test = mean_squared_error(test_y, test_pred)
        var_train = model.score(train_dummies_x, train_y)
        var_test = model.score(test_dummies_x, test_y)
    
    acc_train = accuracy_score(train_y, train_pred_round)
    acc_test = accuracy_score(test_y, test_pred_round)
    f1_train = f1_score(train_y, train_pred_round,average='weighted')
    f1_test = f1_score(test_y, test_pred_round,average='weighted')

    result = {
        'f1_train': f1_train,
        'f1_test': f1_test,
        'acc_train': acc_train,
        'acc_test': acc_test,
    }
    if regression:
        result.update({
            'mse_train': mse_train,
            'mse_test': mse_test,
            'var_train': var_train,
            'var_test': var_test,
        })
    return result

In [10]:
def stats2ftr(stats, how='TOTAL', ftr_name=None):
    ftr = stats.drop(['MONTH','YEAR'], axis=1).groupby('index_seg').sum()
    if how=='TOTAL':
        assert ftr_name is not None, 'please provide the feature name for the aggregated feature'
        ftr = ftr.sum(axis=1).to_frame()
        ftr.columns = [ftr_name]
    ftr = ftr.reset_index()
    return ftr

In [11]:
def get_train_test_dummies(train_x, test_x, fillna_dict):
    train_fillna_x = train_x.fillna(fillna_mode_or_mean)
    test_fillna_x = test_x.fillna(fillna_mode_or_mean)
    train_dummies_x = pd.get_dummies(train_fillna_x)
    test_dummies_x = pd.get_dummies(test_fillna_x)
    for x in list(set(train_dummies_x.columns)-set(test_dummies_x.columns)):
        test_dummies_x[x] = 0
    test_dummies_x = test_dummies_x[train_dummies_x.columns]
    return train_dummies_x, test_dummies_x

# parameters for gridsearchCV
- linear: tol
- ridge: tol, alpha=0.5
- lasso: tol, alpha=0.5
- DTReg: max_depth=10, max_features=None [float, 'sqrt', 'log2']
- DTClsfr: criterion=gini(or entropy), max_features=None [float, 'sqrt', 'log2'], max_depth=None
- RFReg: n_estimators=10, max_features=None [float, 'sqrt', 'log2']
- BagReg: n_estimators=10, max_features=1.0(float or int), max_samples=1.0(float or int)
- AdaReg: n_estimators=50, max_features=None [float, 'sqrt', 'log2'], learning_rate=1.
- GraReg: n_estimators=100, max_features=None [float, 'sqrt', 'log2'], learning_rate=.1, max_depth=3
- logistics: penalty=l2(or l1), C=1.0(like SVM, smaller->stronger regularization), tol=1e-4
- svr: C=1.0, epsilon=0.1, kernal=rbf(linear, poly, sigmoid), degree=3(for poly), epsilon=0.1(C, gamma , epsilon play a very important role in rbf gaussian kernel based SVM ), tol=1e-3

In [12]:
ALPHAs = np.logspace(0,2,10)
DEPTHs = [3, 5, 10, 20, 30, 40, 50]
N_ESTIMATORs = [10, 30, 50, 100, 256, 500]
MAX_FTRs = ['sqrt', 'log2', 0.3, 0.5]
LEARN_RATEs = np.logspace(-3, 1, 6)
PENALTYs = ['l1', 'l2']
TOLs = [1e-2, 1e-3, 1e-4, 1e-5]
GAMMAs = [1e-2, 1e-3, 1e-4, 1e-5]
Cs = np.logspace(-4, 2, 6)
EPs = np.logspace(-4, 2, 6)
DEGs = [2, 3, 4, 5, 6]
CRITERIONs = ['gini', 'entropy']

# lasso and Ridge
params_alpha = {'alpha': ALPHAs}
params_logis = {'C': Cs, 'penalty':PENALTYs}
params_dt_reg = {'max_depth': DEPTHs, 'max_features': MAX_FTRs}
params_dt_cla = {'max_depth': DEPTHs, 'max_features': MAX_FTRs, 'criterion': CRITERIONs}
params_rf_reg = {'n_estimators': N_ESTIMATORs, 'max_features': MAX_FTRs}
params_bag_reg = {'n_estimators': N_ESTIMATORs, 'max_features': MAX_FTRs[2:]}
params_ada_reg = {'n_estimators': N_ESTIMATORs, 'learning_rate': LEARN_RATEs}
params_gra_reg = {'n_estimators': N_ESTIMATORs, 'max_features': MAX_FTRs, 'learning_rate': LEARN_RATEs, 'max_depth':DEPTHs}

params_svm = [
    {'kernel':['linear'], 'C': Cs},
    {'kernel':['rbf'], 'C': Cs, 'gamma': GAMMAs},
    {'kernel':['sigmoid'], 'C': Cs, 'gamma': GAMMAs},
    {'kernel':['poly'], 'C': Cs, 'gamma': GAMMAs, 'degree': DEGs},
]

params_svr = [
    {'kernel':['linear'], 'C': Cs},
    {'kernel':['rbf'], 'C': Cs, 'gamma': GAMMAs},
    {'kernel':['sigmoid'], 'C': Cs, 'gamma': GAMMAs},
    {'kernel':['poly'], 'C': Cs, 'gamma': GAMMAs, 'degree': DEGs},
]


In [19]:
def get_models(no_svm=False):
    """return: model, tuning_params, name, regresion_or_classification
    """
    models = [
        [linear_model.LinearRegression(), {}, "linear", 'regression'],
        [linear_model.Ridge(), params_alpha, "ridge", 'regression'],
        [linear_model.Lasso(), params_alpha, "lasso", 'regression'],
        [tree.DecisionTreeRegressor(), params_dt_reg, "DTReg", 'regression'],
        [ensemble.RandomForestRegressor(), params_rf_reg, "RFReg", 'regression'],
        [ensemble.AdaBoostRegressor(), params_ada_reg, "AdaReg", 'regression'],
        [ensemble.BaggingRegressor(), params_bag_reg, "BagReg", 'regression'],
        [ensemble.GradientBoostingRegressor(), params_gra_reg, "GraReg", 'regression'],
        [linear_model.LogisticRegression(), params_logis, "logistics", 'classification'],
        [tree.DecisionTreeClassifier(), params_dt_cla, "DTClsfr", 'classification'],
        [ensemble.RandomForestClassifier(), params_rf_reg, "RFclsfr", 'classification'],
        [ensemble.AdaBoostClassifier(), params_ada_reg, "Adaclsfr", 'classification'],
        [ensemble.BaggingClassifier(), params_bag_reg, "Bagclsfr", 'classification'],
        [ensemble.GradientBoostingClassifier(), params_gra_reg, "Graclsfr", 'classification'],
    ]
    if not no_svm:
        models+=[[svm.SVR(), params_svr, "SVR", 'regression'],
                 [svm.SVC(), params_svm, "SVM", 'classification']]
    return models

# load raw data

In [154]:
from src.constants import fn_segments_dc, fn_feature_bk_facs_dc, fn_feature_poi_dc, fn_feature_seg_as_edge_dc, fn_feature_seg_as_node_dc
fn_feature_lts_dc = 'feature_lts_dc.csv'
fn_feature_crime_dc = 'feature_crime_incidences_dc.csv'
fn_feature_mov_dc = 'feature_mov_violations_dc.csv'
fn_feature_parking_dc = 'feature_parking_violations_dc.csv'
data_dir = 'data/'

In [155]:
segs = gp.read_file(data_dir+fn_segments_dc)

ftr_segs_col = ['DIRECTIONALITY', 'STREETTYPE', 'SHAPE_Length']
ftr_segs = segs[ftr_segs_col].reset_index()
ftr_segs.columns = ['index_seg']+list(ftr_segs.columns[1:])
ftr_segs.DIRECTIONALITY = ftr_segs.DIRECTIONALITY.apply(lambda x: 'Bi-direction' if x==2 else 'one-way')


In [156]:
ftr_lts = pd.read_csv(data_dir+fn_feature_lts_dc)
ftr_bk = pd.read_csv(data_dir+fn_feature_bk_facs_dc, index_col=0)
ftr_poi = pd.read_csv(data_dir+fn_feature_poi_dc, index_col=0)
ftr_edge = pd.read_csv(data_dir+fn_feature_seg_as_edge_dc)
ftr_node = pd.read_csv(data_dir+fn_feature_seg_as_node_dc)

In [157]:
crime_stats = pd.read_csv(data_dir+fn_feature_crime_dc)
mov_stats = pd.read_csv(data_dir+fn_feature_mov_dc)
parking_stats = pd.read_csv(data_dir+fn_feature_parking_dc)

ftr_mov = stats2ftr(mov_stats, how='NOT_TOTAL')
ftr_crime = stats2ftr(crime_stats, how='NOT_TOTAL')
ftr_parking = stats2ftr(parking_stats, how='NOT_TOTAL')

ftr_mov_total = stats2ftr(mov_stats, ftr_name='moving_violations_cnt')
ftr_crime_total = stats2ftr(crime_stats, ftr_name='crime_incidents_cnt')
ftr_parking_total = stats2ftr(parking_stats, ftr_name='parking_violations_cnt')

In [158]:
cut_off = 100
dfs = [ftr_lts, ftr_segs, ftr_bk, ftr_poi, ftr_edge, ftr_node, ftr_mov, ftr_crime, ftr_parking]
df = reduce(lambda left,right: pd.merge(left,right, how='left'), dfs)
df = df[~df.LTS.isnull()&(df.LTS!=10)]
df.drop('index_seg', axis=1, inplace=True)
non_na_count = df.count()
keep_col = non_na_count[non_na_count>cut_off].index
df = df[keep_col]
df.shape

(13293, 184)

# get train and test

In [159]:
train = df.sample(frac=0.8, random_state=1)
test = df[~df.index.isin(train.index)]
train.shape, test.shape

((10634, 184), (2659, 184))

In [160]:
train_y = train.LTS.values
train_x = train.drop('LTS', axis=1)
test_y = test.LTS.values
test_x = test.drop('LTS', axis=1)

In [161]:
train_x.shape, test_x.shape

((10634, 183), (2659, 183))

In [162]:
# fillna_mode_or_mean = train_x[['cycle_lane', 'cycle_way', 'side_walk', 'bikable']].mode().T.to_dict()[0]
fillna_mode_or_mean = train_x[['cycle_lane', 'side_walk', 'bikable', 'DIRECTIONALITY', 'STREETTYPE']].mode().T.to_dict()[0]
fillna_mode_or_mean.update(train_x.mean().to_dict())

best with f1 test
- ftr_bk, ftr_poi, ftr_edge, ftr_node:   DTClsfr->0.486618
- ftr_bk, ftr_poi, ftr_edge, ftr_node, ftr_mov_total, ftr_crime_total(one aggregate column): DTClsfr->0.479885
- ftr_bk, ftr_poi, ftr_edge, ftr_node, ftr_mov, ftr_crime, cut_off = 100: DTClsfr->0.495041
- ftr_segs, ftr_bk, ftr_poi, ftr_edge, ftr_node, ftr_mov, ftr_crime, cut_off=100: RFReg->0.569317
- ftr_segs, ftr_bk, ftr_poi, ftr_edge, ftr_node, ftr_mov, ftr_crime, ftr_parking, cut_off=100: BagReg->0.571100 or sometimes RFREG->0.569


In [140]:
train_dummies_x, test_dummies_x = get_train_test_dummies(train_x, test_x, fillna_mode_or_mean)
print 'dummies features =', train_dummies_x.shape

models = get_models(no_svm=False)
results = {}
for name, model, is_regression in models:
    print('model = %s' % name)
    results[name] = fit_pred(model, train_dummies_x, train_y, test_dummies_x, test_y, regression=is_regression)

df_results = pd.DataFrame(results)
df_results.T.sort('f1_test', ascending=False)

dummies features = (10634, 210)
model = linear
model = ridge
model = lasso
model = DTReg
model = RFReg
model = AdaReg
model = BagReg
model = GraReg
model = logistics
model = DTClsfr
model = SVR
model = SVM


,acc_test,acc_train,f1_test,f1_train,mse_test,mse_train,var_test,var_train
RFReg,0.555848,0.888095,0.569198,0.890695,0.557774,0.095528,0.522862,0.918518
BagReg,0.540429,0.891198,0.552007,0.893558,0.553546,0.094992,0.526478,0.918975
DTReg,0.541181,0.644536,0.546847,0.649364,0.695903,0.423791,0.404702,0.638519
DTClsfr,0.538548,1.000000,0.538710,1.000000,NaN,NaN,NaN,NaN
GraReg,0.523881,0.545138,0.534443,0.557736,0.537734,0.483060,0.540004,0.587964
linear,0.422339,0.435396,0.428018,0.435718,0.727979,0.695277,0.377263,0.406950
ridge,0.403159,0.407467,0.402802,0.402541,0.748217,0.714013,0.359951,0.390969
logistics,0.446408,0.443013,0.306645,0.300528,NaN,NaN,NaN,NaN
SVM,0.430613,0.993323,0.262058,0.993310,NaN,NaN,NaN,NaN
AdaReg,0.288454,0.295467,0.176600,0.178236,0.809287,0.793741,0.307709,0.322963


best F1
- ftr_segs, ftr_bk, ftr_poi, ftr_edge, ftr_node, ftr_mov, ftr_crime, ftr_parking, cut_off=100: RFReg->0.553822

In [141]:
fillna_unknown_or_zero = train_x.dtypes.apply(lambda x: 'unknown' if x.name=='object' else 0.0).to_dict()

In [143]:
train_dummies_x, test_dummies_x = get_train_test_dummies(train_x, test_x, fillna_unknown_or_zero)
print 'dummies features =', train_dummies_x.shape


models = get_models()
results = {}
for name, model, is_regression in models:
    print('model = %s' % name)
    results[name] = fit_pred(model, train_dummies_x, train_y, test_dummies_x, test_y, regression=is_regression)

df_results = pd.DataFrame(results)
df_results.T.sort('f1_test', ascending=False)

dummies features = (10634, 210)
model = linear
model = ridge
model = lasso
model = DTReg
model = RFReg
model = AdaReg
model = BagReg
model = GraReg
model = logistics
model = DTClsfr
model = SVR
model = SVM


,acc_test,acc_train,f1_test,f1_train,mse_test,mse_train,var_test,var_train
SVR,0.253855,0.858379,0.139016,0.807684,1.166471,0.172517,0.002162,0.852848
lasso,0.253479,0.266033,0.150752,0.158650,0.973118,0.976800,0.167562,0.166820
AdaReg,0.280557,0.286064,0.172581,0.173725,0.814547,0.797884,0.303209,0.319430
SVM,0.430613,0.993323,0.262058,0.993310,NaN,NaN,NaN,NaN
logistics,0.446408,0.443013,0.306645,0.300528,NaN,NaN,NaN,NaN
ridge,0.403159,0.407467,0.402802,0.402541,0.748217,0.714013,0.359951,0.390969
linear,0.422339,0.435396,0.428018,0.435718,0.727979,0.695277,0.377263,0.406950
GraReg,0.523505,0.545138,0.534114,0.557736,0.538250,0.483060,0.539563,0.587964
DTReg,0.538548,0.645759,0.543772,0.650608,0.712428,0.423510,0.390565,0.638759
DTClsfr,0.547574,1.000000,0.546798,1.000000,NaN,NaN,NaN,NaN
